In [2]:
import sys
import os

src_path = os.environ["src_path"]
sys.path.append(src_path)



In [3]:
from datetime import timedelta, timezone, datetime

import hopsworks
import pandas as pd
import numpy as np
import src.components.feature_group_config as config
from src.components.feature_group_config import FEATURE_VIEW_METADATA

In [4]:
current_date = pd.to_datetime(datetime.utcnow(), utc=True).floor('h')
current_date

/Users/sanjaydevarajan/anaconda3/bin/python


In [ ]:
# load recent batch of data from feature store
from src.components.inference import load_batch_of_features_from_store
features = load_batch_of_features_from_store(current_date)

In [ ]:
features

In [ ]:
from src.components.inference import load_model_from_registry, get_model_predictions

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

In [ ]:
predictions['date'] = current_date
predictions

In [ ]:
from src.plot import plot_one_sample

plot_one_sample(example_id=6,
                features=features,
                targets=predictions['prediction_demand'],
                predictions=pd.Series(predictions['predicted_demand']))

In [ ]:
from src.plot import plot_one_sample

plot_one_sample(example_id=8,
                features=features,
                targets=predictions['prediction_demand'],
                predictions=pd.Series(predictions['predicted_demand']))

In [ ]:
# save predictions to the feature store
from src.components.feature_store_api import get_feature_store
import src.components.feature_group_config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name = config.FEATURE_GROUP_PREDICTIONS_METADATA.name,
    version = 1,
    description = "Predictions generate by oru production model",
    primary_key = ["sub_region_code", "date"],
    event_time = "date"
)

In [ ]:
feature_group.insert(predictions,write_options={"wait_for_job": False})